In [1]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.plotly import chartFigure
from simple.funcs import symlog

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

2224

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 0, 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 1, 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2, 2636., 96.),
           ('2022-05-08T00:01:00.668000', 3, 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 4, 2636.,  2.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [5]:
len(T), len(S)

(2224, 1701)

In [6]:
# timed chart
fig = chartFigure(Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3),
            Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3),
            Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
            Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines')
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'size': array([ 7.58…

In [7]:
C = renko(T, 0.5)
C[:5]

array([[   0. ,   15. , 2635.5, 2636. ],
       [  16. ,   44. , 2636.5, 2636. ],
       [  45. ,  207. , 2635.5, 2636. ],
       [ 208. ,  239. , 2636.5, 2636. ],
       [ 240. ,  240. , 2635.5, 2636. ]])

In [8]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 1},
     'opacity': 0.2,
     'type': 'rect',
     'x0': str(T.DateTimeA[int(c[0])]),
     'x1': str(T.DateTimeA[int(c[1])]),
     'y0': c[2],
     'y1': c[3]
 } for c in C]